In [1]:
import numpy as np
from scipy import *
from pylab import *
import os
import pickle

%matplotlib inline

In [2]:
folder_Pk = '/global/cscratch1/sd/jialiu/CAMELS/Pk/' ### original, unorganized Pk
home = '/global/homes/j/jialiu/cosmo_dep_feedback/'

###### the simulation we operate with
isim = 'IllustrisTNG'

In [3]:
### fixed parameters: 
Omega_b = 0.049
h = 0.6711
ns = 0.9624
w = -1
mnu = 0.0 #eV
Omega_k = 0.0

In [18]:
#### Omega_m sigma_8 A1 A2 A3 A4
sim_name = np.loadtxt(home+'params/CosmoAstroSeed_%s.txt'%(isim), usecols=(0), dtype='U')
sim_params = np.loadtxt(home+'params/CosmoAstroSeed_%s.txt'%(isim), usecols=range(1,7))

#### change all the baryon parameters to log
labels=['Omega_m', 'sigma_8', 'log10(A1)', 'log10(A2)', 'log10(A3)', 'log10(A4)']
sim_params = concatenate([sim_params[:,:2], log10(sim_params[:,2:])],axis=1)

sim_name, sim_params

(array(['LH_0', 'LH_1', 'LH_2', ..., 'CV_24', 'CV_25', 'CV_26'],
       dtype='<U7'),
 array([[ 0.309     ,  0.979     ,  0.49308703,  0.04996963, -0.17489859,
         -0.27423533],
        [ 0.3026    ,  0.9394    ,  0.53402738,  0.59784541,  0.01414657,
          0.06471998],
        [ 0.4282    ,  0.753     , -0.15111534, -0.4268614 ,  0.2929026 ,
         -0.20259395],
        ...,
        [ 0.3       ,  0.8       ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.3       ,  0.8       ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.3       ,  0.8       ,  0.        ,  0.        ,  0.        ,
          0.        ]]))

In [17]:
idx_LH = where(np.char.find(sim_name, 'LH')!=-1)
idx_1P = where(np.char.find(sim_name, '1P')!=-1)
idx_CV = where(np.char.find(sim_name, 'CV')!=-1)

In [ ]:
lhc_params = sim_params[idx_LH]
f, ax=subplots(1,5, figsize=(14,3))

for i in range(5):
    ax[i].scatter(lhc_params.T[i], lhc_params.T[i+1], color='b', label=isim,s=5)
    ax[i].set_xlabel(labels[i])
    ax[i].set_ylabel(labels[i+1])   
    ax[i].legend()
plt.tight_layout()
show()